## 1. Understand the dataset

## 2. Combine all dataset

In [47]:
import os

# needs openpyxl
import pandas as pd

excel_data: list[pd.DataFrame] = []
csv_data: list[pd.DataFrame] = []

for root, dirs, files in os.walk('./pre_test'):
    for file in files:
        path = os.path.join(root, file)
        if file.endswith('.xlsx'):
            excel_data.append(pd.read_excel(path))
        elif file.endswith('.csv'):
            csv_data.append(pd.read_csv(path))

In [48]:
for data_frame in [*excel_data, *csv_data]:
    data_frame.drop(data_frame[-1:0].index, inplace=True)

In [49]:
data = pd.concat([*excel_data, *csv_data])

## 3. Check and change data type of each column to appropriate type
## 4. Replace ( – ) by 0

In [50]:
# print(data.dtypes)

for c in data.columns:
    if c == 'Grade/100.00' or c.startswith('Q.'):
        data[c] = data[c].map(lambda x: float(x) if x != '-' else 0)

## 5. Drop duplicates

In [51]:
data = data.drop_duplicates()

## 6. Create new column ‘Label’ – Passed if greater than or equal to 60, Failed if less than 60

In [ ]:

data['Label'] = data['Grade/100.00'].map(
    lambda x: 'Passed' if x >= 60 else 'Failed'
)

## 7. Rename Grade/100.00 to Total Score

In [ ]:
data = data.rename(columns={'Grade/100.00': 'Total Score'})

## 8. Detect outliers in Total Score column – Keep outliers in our case

In [ ]:
q1 = data['Total Score'].quantile(0.25)
q3 = data['Total Score'].quantile(0.75)

iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

outliers = data[
    (data['Total Score'] < lower_bound) | (data['Total Score'] > upper_bound)
]

outliers

## 9. Drop row where State is not Finished

In [55]:
data = data.drop(data.loc[data['State'] != "Finished"].index)
data = data.reindex()

## 10. Deal with missing values if any

In [56]:
print(len(data))
data.dropna()
print(len(data))

2670
2670


In [57]:
data.to_excel('pre_test_output.xlsx')